# Sentiment analysis with Logistic Regression

## Logistic Regression

> Some of the explanations in this tutoarial are inspired by chapters 3 and 8 in the book _"Python Machine Learning"_ by Sebastian Raschka

First, let's introduce what Logistic Regression is. 

[Logistic Regression](https://en.wikipedia.org/wiki/Logistic_regression) is a classification model that is very easy to implement and performs very well on linearly separable classes. It is one of the most widely used
algorithms for classification in industry too, which makes it attractive to play with.

_Very_ simplistically explained, Logistic Regression works as follows:

![Logistic Regression](./images/logistic_regression.png)

First we will define the input for our algorithm. The imput will be each sample in whatever dataset we are working with. Each sample will consist of several features. For example, if we're working with housing price prediction, the features for each sample could be the size of the house, number of rooms, etc. We'll call the input vector **X**.

For the algorythm to learn, we need to define variables that we can adjust accordingly to what we want to predict. We will create a vector of _weights_ (**W**) that the model will adjust in order to predict more accurately. The process of adjusting those weights is what we call **learning**.

For every input sample, we will perform a dot product of the features by the weights **XW**. This product is sometimes referred as _net input_. This will give us a real number. Since in this particular problem we want to _classify_ (positive/negative), we need squash this number in the range [0, 1]. This will give us the _probability_ of a positive event. A function that does precisely that is called **sigmoid**. The sigmoid function looks like this:

![sigmoid](./images/sigmoid.svg)

What sigmoid is doing is basically transforming big inputs into a value close to 1, and small inputs into a value close to 0. This is exactly what we want. 

We will do this for every sample in our training set and compute the errors. To calculate the error we only need to compare our prediction with the true label for each sample. We will sum the square errors of all the samples to get a global prediction error. This will be our **cost function**.

A cost function is then something we want to minimize. **Gradient descent** is a method for finding the minimum of a function of multiple variables, such like the one we're dealing with here.

### Gradient descent

> Watch [this](https://www.youtube.com/watch?v=IHZwWFHWa-w) video for a visual introduction to Gradient Descent

Once all our training samples have been computed and the error calculated with our cost function, we need to _minimize_ that cost function. A method for doing that is gradient descent. There are many [articles](https://en.wikipedia.org/wiki/Gradient_descent) that contain detailed explanations _and_ implementations of GD, so let's not do this here. However is good to have an intuition.

For illustration purposes, let's think about a function with two parameters. Something like this one:

![Gradient Descent](./images/Gradient_descent.png)

Gradient descent will try to find the minimum of the function. To do so, we calculate the slope of the function at a certain point, and move towards the direction that makes the function decrease. There are some things to have in mind though.

As you can see a function can have one or several _local minimum_. In a local minimum, the slope will be zero and GD will "think" it's found the global minimum. To avoid this, we can choose a bigger "step" when we move towards the minimum. The "size" of the step towards the minimun is what we call the **learning rate**, and it's another adjustable parameter.

We need to be careful here: If we choose a too small learning rate, we can get stuck in a local minimum. If we instead choose a too big learning rate, we risk overshooting the global minimum. We need to experiment, and the adecuate learning rate depends on the particular problem and the data.

### The training process

In order for logistic regression to learn, we need to repeat the process descrived before several times. Each one of these times is called an **epoch**. The number of epochs to run depends on the problem and the training data. It is... yes, another tunnable parameter of the algorithm.

The set of all tunnable parameters is called **hyperparameters** of the model.

Like with the leatning rate, we need to be careful when choosing the number of epochs: If we train too many epochs, we risk **overfitting**. This means that our model will "memorize" the training data and will generalize badly when presented new data. 

If we train too little, it will fail to find any pattern and the prediction accuracy will be very low. This is known as **underfitting**.

There are techniques that help prevent overfitting. These **regularization** techniques are out of the scope of this tutorial, but... guess! It's also something to tune and experiment with :)

This is why when training a model you need to set aside a _test dataset_ in order to know the accuracy of your algorithm in unknown data. The test dataset will **never** be used during training

## Sentiment analysis

Let's first of all have a look at the data

In [81]:
import pandas as pd

#Load the data into a DataFrame
train = pd.read_csv('../pytorch-text-classification/data/Q1-yesno-elno_cleaned_data_current/trainval.csv', encoding='latin-1',sep=',')
test = pd.read_csv('../pytorch-text-classification/data/Q1-yesno-elno_cleaned_data_current/test.csv', encoding='latin-1',sep=',')

train.head(10)

,recipedetails,label
0,pure soup batch blender use immers hand blende...,yes
1,use bread flour semolina flour turn dough oil ...,no
2,note author say 1 tsp dri mint could use dri m...,no
3,let cool complet store airtight contain keep f...,yes
4,add veget stir fri minut two,yes
5,shrimp bask cornmeal focu grit,no
6,leftov fill wa done ad sauc add broth leftov f...,no
7,preheat oven 375f,no
8,fri onion garlic larg pot add splash water pre...,yes
9,add yogurt,no


In [82]:
train.recipedetails

0       pure soup batch blender use immers hand blende...
1       use bread flour semolina flour turn dough oil ...
2       note author say 1 tsp dri mint could use dri m...
3       let cool complet store airtight contain keep f...
4                            add veget stir fri minut two
5                          shrimp bask cornmeal focu grit
6       leftov fill wa done ad sauc add broth leftov f...
7                                       preheat oven 375f
8       fri onion garlic larg pot add splash water pre...
9                                              add yogurt
10      uncov bake sauc 375 f oven brown 45 minut one ...
11      cook noodl boil water 3 min ceas stick togeth ...
12      put freezer minimum 2 month mix fresh tonic fr...
13      note ensur maximum light allow dough ferment p...
14          fill glass one third full add water crush ice
15      preheat oven 350 degre f greas flour 9 x5 loaf...
16                               spiral zucchini set asid
17      grill 

As we can see, the structure of a twit varies a lot between twit and twit. They have different lengths, letters, numbers, extrange characters, etc. 

It is also important to note that **a lot** of words are not correctly spelled, for example the word _"Juuuuuuuuuuuuuuuuussssst"_ or the word _"frie"_ instear of _"friend"_

This makes it hard to mesure how positive or negative are the words withing the corpus of twits. If they were all correct dictionary words, we could use a **lexicon** to punctuate words. However because of the nature of social media language, we cannot do that. 

So we need a way of scoring the words such that words that appear in positive twits have greater score that those that appear in negative twits.

But first... how do we represent the twits as vectors we can input to our algorithm?

### Bag of words

One thing we could do to represent the twits as equal-sized vectors of numbers is the following:

* Create a list (vocabulary) with all the unique words in the whole corpus of twits. 
* We construct a feature vector from each twit that contains the counts of how often each word occurs in the particular twit

_Note that since the unique words in each twit represent only a small subset of all the words in the bag-of-words vocabulary, the feature vectors will mostly consist of zeros_

Let's construct the bag of words. We will work with a smaller example for illustrative purposes, and at the end we will work with our real data.

In [83]:
from sklearn.feature_extraction.text import CountVectorizer

twits = [
    'This is amazing!',
    'ML is the best, yes it is',
    'I am not sure about how this is going to end...'
]

count = CountVectorizer()
bag = count.fit_transform(twits)

count.vocabulary_

{'this': 13,
 'is': 7,
 'amazing': 2,
 'ml': 9,
 'the': 12,
 'best': 3,
 'yes': 15,
 'it': 8,
 'am': 1,
 'not': 10,
 'sure': 11,
 'about': 0,
 'how': 6,
 'going': 5,
 'to': 14,
 'end': 4}

As we can see from executing the preceding command, the vocabulary is stored in a Python dictionary that maps the unique words to integer indices. Next, let's print the feature vectors that we just created:

In [84]:
bag.toarray()

array([[0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 2, 1, 1, 0, 0, 1, 0, 0, 1],
       [1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0]], dtype=int64)

Each index position in the feature vectors corresponds to the integer values that are stored as dictionary items in the CountVectorizer vocabulary. For example, the first feature at index position 0 resembles the count of the word 'about' , which only occurs in the last document, and the word 'is' , at index position 7, occurs in all three twits (two times in the second twit). These values in the feature vectors are also called the **raw term frequencies**: `tf(t,d )` —the number of times a term `t` occurs in a document `d`.

### How relevant are words? Term frequency-inverse document frequency

We could use these raw term frequencies to score the words in our algorithm. There is a problem though: If a word is very frequent in _all_ documents, then it probably doesn't carry a lot of information. In order to tacke this problem we can use **term frequency-inverse document frequency**, which will reduce the score the more frequent the word is accross all twits. It is calculated like this:

\begin{equation*}
tf-idf(t,d) = tf(t,d) ~ idf(t,d)
\end{equation*}

_tf(t,d)_ is the raw term frequency descrived above. _idf(t,d)_ is the inverse document frequency, than can be calculated as follows:

\begin{equation*}
\log \frac{n_d}{1+df\left(d,t\right)}
\end{equation*}

where `n` is the total number of documents and _df(t,d)_ is the number of documents where the term `t` appears. 

The `1` addition in the denominator is just to avoid zero term for terms that appear in all documents. Ans the `log` ensures that low frequency term don't get too much weight.

Fortunately for us `scikit-learn` does all those calculations for us:

In [85]:
import numpy as np

from sklearn.feature_extraction.text import TfidfTransformer

tfidf = TfidfTransformer(use_idf=True,
                         norm='l2',
                         smooth_idf=True)

np.set_printoptions(precision=2)

# Feed the tf-idf transformer with our previously created Bag of Words
tfidf.fit_transform(bag).toarray()

array([[0.  , 0.  , 0.72, 0.  , 0.  , 0.  , 0.  , 0.43, 0.  , 0.  , 0.  ,
        0.  , 0.  , 0.55, 0.  , 0.  ],
       [0.  , 0.  , 0.  , 0.4 , 0.  , 0.  , 0.  , 0.47, 0.4 , 0.4 , 0.  ,
        0.  , 0.4 , 0.  , 0.  , 0.4 ],
       [0.33, 0.33, 0.  , 0.  , 0.33, 0.33, 0.33, 0.2 , 0.  , 0.  , 0.33,
        0.33, 0.  , 0.25, 0.33, 0.  ]])

As you can see, words that appear in all documents like _is_ (with 0.47 ), get a lower score than others that don't appear in all documents, like _amazing_ (with 0.72).

Note also that `norm='l2'` parameter: This is an important one, and what is doing is normalize the tf-idfs so that they're all in the same scale and thus work better with Logistic Regression.

## Data clean up (yay...)

### Removing stop words

Now that we know how to format and score our input, we can start doing the analysis! Can we?... Well, we _can_, but let's look at our **real** vocabulary. Specifically, the most common words:

In [86]:
from collections import Counter

vocab = Counter()
for twit in train.recipedetails:
    for word in twit.split(' '):
        vocab[word] += 1

vocab.most_common(20)

[('add', 808),
 ('minut', 658),
 ('heat', 432),
 ('cook', 432),
 ('1', 417),
 ('2', 335),
 ('stir', 334),
 ('bake', 333),
 ('water', 321),
 ('mix', 314),
 ('place', 313),
 ('pan', 312),
 ('serv', 306),
 ('bowl', 301),
 ('mixtur', 296),
 ('oil', 287),
 ('oven', 259),
 ('remov', 255),
 ('salt', 247),
 ('butter', 234)]

As you can see, the most common words are meaningless in terms of sentiment: _I, to, the, and_... they don't give any information on positiveness or negativeness. They're basically **noise** that can most probably be eliminated. Let's see the whole distribution to convince ourselves of this:

In [87]:
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [88]:
import math

def plot_distribution(vocabulary):

    hist, edges = np.histogram(list(map(lambda x:math.log(x[1]),vocabulary.most_common())), density=True, bins=500)

    p = figure(tools="pan,wheel_zoom,reset,save",
               toolbar_location="above",
               title="Word distribution accross all twits")
    p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color="#555555", )
    show(p)

plot_distribution(vocab)

It's clear now that a porcion of the words are overly represented. These kind of words are called _stop words_, and it is a common practice to remove them when doing text analysis. Let's do it and see the distribution again:

In [89]:
import nltk

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ehosseiniasl/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [90]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

vocab_reduced = Counter()
for w, c in vocab.items():
    if not w in stop:
        vocab_reduced[w]=c

vocab_reduced.most_common(20)

[('add', 808),
 ('minut', 658),
 ('heat', 432),
 ('cook', 432),
 ('1', 417),
 ('2', 335),
 ('stir', 334),
 ('bake', 333),
 ('water', 321),
 ('mix', 314),
 ('place', 313),
 ('pan', 312),
 ('serv', 306),
 ('bowl', 301),
 ('mixtur', 296),
 ('oil', 287),
 ('oven', 259),
 ('remov', 255),
 ('salt', 247),
 ('butter', 234)]

In [91]:
stop

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

This looks better, only in the 20 most common words we already see words that make sense: _good, love, really_... Let's see the distribution now

In [92]:
plot_distribution(vocab_reduced)

### Removing special characters and "trash"

We still se a very uneaven distribution. If you look closer, you'll see that we're also taking into consideration punctuation signs ('-', ',', etc) and other html tags like `&amp`. We can definitely remove them for the sentiment analysis, but we will try to keep the emoticons, since those _do_ have a sentiment load:

In [93]:
import re

def preprocessor(text):
    """ Return a cleaned version of text
    """
    # Remove HTML markup
    text = re.sub('<[^>]*>', '', text)
    # Save emoticons for later appending
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    # Remove any non-word character and append the emoticons,
    # removing the nose character for standarization. Convert to lower case
    text = (re.sub('[\W]+', ' ', text.lower()) + ' ' + ' '.join(emoticons).replace('-', ''))
    
    return text

print(preprocessor('This!! twit man :) is <b>nice</b>'))

this twit man is nice :)


We are almost ready! There is another trick we can use to reduce our vocabulary and consolidate words. If you think about it, words like: love, loving, etc. _Could_ express the same positivity. If that was the case, we would be  having two words in our vocabulary when we could have only one: lov. This process of reducing a word to its root is called **steaming**.

We also need a _tokenizer_ to break down our twits in individual words. We will implement two tokenizers, a regular one and one that does steaming:

In [94]:
from nltk.stem import PorterStemmer

porter = PorterStemmer()

def tokenizer(text):
    return text.split()

def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

print(tokenizer('Hi there, I am loving this, like with a lot of love'))
print(tokenizer_porter('Hi there, I am loving this, like with a lot of love'))

['Hi', 'there,', 'I', 'am', 'loving', 'this,', 'like', 'with', 'a', 'lot', 'of', 'love']
['Hi', 'there,', 'I', 'am', 'love', 'this,', 'like', 'with', 'a', 'lot', 'of', 'love']


## Training Logistic Regression

We are finally ready to train our algorythm. We need to choose the best hyperparameters like the _learning rate_ or _regularization strength_. We also would like to know if our algorithm performs better steaming words or not, or removing html or not, etc...

To take these decisions methodically, we can use a Grid Search. Grid search is a method of training an algorythm with different variations of parameters to latter select the best combination

In [95]:
from sklearn.model_selection import train_test_split

# split the dataset in train and test
X = train['recipedetails']
y = train['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0, stratify=y)

In the code line above, `stratify` will create a train set with the same class balance than the original set

In [96]:
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import BaggingRegressor

tfidf = TfidfVectorizer(strip_accents=None,
                        lowercase=False,
                        preprocessor=None)

param_grid = [{'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'vect__preprocessor': [None, preprocessor],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]},
              {'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'vect__preprocessor': [None, preprocessor],
               'vect__use_idf':[False],
               'vect__norm':[None],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]},
              ]

lr_tfidf = Pipeline([('vect', tfidf),
                     ('clf', LogisticRegression(random_state=0))])

#lr_tfidf = Pipeline([('vect', tfidf),
#                     ('clf', BaggingRegressor(LogisticRegression(random_state=0), n_estimators=50, bootstrap=True))])

gs_lr_tfidf = GridSearchCV(lr_tfidf, param_grid,
                           scoring='accuracy',
                           cv=5,
                           verbose=1,
                           n_jobs=-1)

In [67]:
# Note: This may take a long while to execute, like... 1 or 2 hours
gs_lr_tfidf.fit(X_train, y_train)

Fitting 5 folds for each of 96 candidates, totalling 480 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    3.6s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:   35.5s
[Parallel(n_jobs=-1)]: Done 418 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 480 out of 480 | elapsed:  1.7min finished
/home/ehosseiniasl/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=False, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,...e, penalty='l2', random_state=0, solver='warn',
          tol=0.0001, verbose=0, warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid=[{'vect__ngram_range': [(1, 1)], 'vect__stop_words': [['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's...se_idf': [False], 'vect__norm': [None], 'clf__penalty': ['l1', 'l2'], 'clf__C': [1.0, 10.0, 100.0]}],
       pre_dispatch='2*n_jobs', refit=T

In [26]:
print('Best parameter set: ' + str(gs_lr_tfidf.best_params_))
print('Best accuracy: %.3f' % gs_lr_tfidf.best_score_)

Best parameter set: {'clf__C': 100.0, 'clf__penalty': 'l1', 'vect__ngram_range': (1, 1), 'vect__preprocessor': None, 'vect__stop_words': ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how',

Interestingly, the set of parameters that best results give us are:

* A regularization strength of `1.0` using `l2` regularization
* Using our `preprocessor` (removing html, keeping emoticons, etc) _does_ improve the performance
* Surprisingly, removing stop words does not improve accuracy
* word steming doesn't seem to help either

As youcan see, sometimes intuition may lead to wrong decisions, and it's important to _test_ all our assumptions. 

Let's see what's our best accuracy then:

In [27]:
clf = gs_lr_tfidf.best_estimator_
print('Accuracy in test: %.3f' % clf.score(X_test, y_test))

Accuracy in test: 0.946


In [97]:
for _ in range(5):
    gs_lr_tfidf.fit(X_train, y_train)
    clf = gs_lr_tfidf.best_estimator_
    print('Accuracy in test: %.3f' % clf.score(X_test, y_test))

Fitting 5 folds for each of 96 candidates, totalling 480 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.
[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    2.1s
[Parallel(n_jobs=-1)]: Done 168 tasks      | elapsed:    9.5s
[Parallel(n_jobs=-1)]: Done 418 tasks      | elapsed:   20.5s
[Parallel(n_jobs=-1)]: Done 480 out of 480 | elapsed:   23.4s finished
/home/ehosseiniasl/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'ani', 'becaus', 'befor', 'doe', 'dure', 'ha', 'hi', "it'", 'onc', 'onli', 'ourselv', "she'", "should'v", 'themselv', 'thi', 'veri', 'wa', 'whi', "you'r", "you'v", 'yourselv'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
/home/ehosseiniasl/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  Fut

Accuracy in test: 0.554
Fitting 5 folds for each of 96 candidates, totalling 480 fits


[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Done 305 tasks      | elapsed:   13.7s
[Parallel(n_jobs=-1)]: Done 480 out of 480 | elapsed:   21.9s finished
/home/ehosseiniasl/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'ani', 'becaus', 'befor', 'doe', 'dure', 'ha', 'hi', "it'", 'onc', 'onli', 'ourselv', "she'", "should'v", 'themselv', 'thi', 'veri', 'wa', 'whi', "you'r", "you'v", 'yourselv'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
/home/ehosseiniasl/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.


Accuracy in test: 0.554
Fitting 5 folds for each of 96 candidates, totalling 480 fits


...

[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Done 303 tasks      | elapsed:   13.9s
[Parallel(n_jobs=-1)]: Done 480 out of 480 | elapsed:   21.9s finished
/home/ehosseiniasl/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'ani', 'becaus', 'befor', 'doe', 'dure', 'ha', 'hi', "it'", 'onc', 'onli', 'ourselv', "she'", "should'v", 'themselv', 'thi', 'veri', 'wa', 'whi', "you'r", "you'v", 'yourselv'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
/home/ehosseiniasl/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 16 concurrent workers.


Accuracy in test: 0.554
Fitting 5 folds for each of 96 candidates, totalling 480 fits


[Parallel(n_jobs=-1)]: Done  18 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Done 307 tasks      | elapsed:   14.0s
[Parallel(n_jobs=-1)]: Done 480 out of 480 | elapsed:   22.0s finished
/home/ehosseiniasl/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'ani', 'becaus', 'befor', 'doe', 'dure', 'ha', 'hi', "it'", 'onc', 'onli', 'ourselv', "she'", "should'v", 'themselv', 'thi', 'veri', 'wa', 'whi', "you'r", "you'v", 'yourselv'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


Accuracy in test: 0.554


/home/ehosseiniasl/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


If we would like to use the classifier in another place, or just not train it again and again everytime, we can save the model in a pickle file:

In [28]:
import pickle
import os

pickle.dump(clf, open(os.path.join('data', 'logisticRegression.pkl'), 'wb'), protocol=4)

Finally, let's run some tests :-)

In [29]:
twits = [
    "This is really bad, I don't like it at all",
    "I love this!",
    ":)",
    "I'm sad... :("
]

preds = clf.predict(twits)

for i in range(len(twits)):
    print(f'{twits[i]} --> {preds[i]}')

This is really bad, I don't like it at all --> no
I love this! --> no
:) --> no
I'm sad... :( --> no


## And you're done! I hope you liked this!